# First Order Logic

In [1]:
class UnificationError(Exception):
    pass
def unify(term1, term2, substitution=None, level=0):
    if substitution is None:
        substitution = {}
    indent = "  " * level 
    print(f"{indent}Unifying: {term1} and {term2}")
    if term1 == term2:
        print(f"{indent}Terms are identical, no substitution.")
        return substitution
    elif is_variable(term1):
        print(f"{indent}Unifying variable {term1} with {term2}")
        return unify_variable(term1, term2, substitution, level + 1)
    elif is_variable(term2):
        print(f"{indent}Unifying variable {term2} with {term1}")
        return unify_variable(term2, term1, substitution, level + 1)
    elif isinstance(term1, tuple) and isinstance(term2, tuple):
        if term1[0] != term2[0]: 
            raise UnificationError(f"Cannot unify {term1} with {term2}, different function names.")
        for t1, t2 in zip(term1[1], term2[1]):
            substitution = unify(t1, t2, substitution, level + 1)
        return substitution
    else:
        raise UnificationError(f"Cannot unify {term1} with {term2}, they are incompatible.")
def unify_variable(var, term, substitution, level):
    indent = "  " * level  
    if var in substitution:
        print(f"{indent}Variable {var} is already substituted as {substitution[var]}")
        return unify(substitution[var], term, substitution, level + 1)
    if term == var:
        print(f"{indent}Variable {var} is the same as the term, no substitution.")
        return substitution
    if is_variable(term) and var in get_variables(term):
        raise UnificationError(f"{indent}Cannot unify variable {var} with {term} (circular unification).")
    print(f"{indent}Substituting {var} with {term}")
    substitution[var] = term
    return substitution
def is_variable(term):
    return isinstance(term, str) and term.islower()
def get_variables(term):
    if is_variable(term):
        return {term}
    elif isinstance(term, tuple):
        variables = set()
        for arg in term[1]:
            variables.update(get_variables(arg))
        return variables
    else:
        return set()
if __name__ == "__main__":
    print("\nExample 1: Unifying P(x, a, b) and P(y, z, b)\n")
    term1 = ('P', ['x', 'a', 'b'])
    term2 = ('P', ['y', 'z', 'b'])
    try:
        substitution = unify(term1, term2)
        print("\nFinal Unification Result:", substitution)
    except UnificationError as e:
        print("Unification failed:", e)
    print("\nExample 2: Unifying P(x, f(Y)) and P(Z, f(a))\n")
    term3 = ('P', ['x', ('f', ['Y'])])
    term4 = ('P', ['Z', ('f', ['a'])])
    try:
        substitution = unify(term3, term4)
        print("\nFinal Unification Result:", substitution)
    except UnificationError as e:
        print("Unification failed:", e)



Example 1: Unifying P(x, a, b) and P(y, z, b)

Unifying: ('P', ['x', 'a', 'b']) and ('P', ['y', 'z', 'b'])
  Unifying: x and y
  Unifying variable x with y
    Substituting x with y
  Unifying: a and z
  Unifying variable a with z
    Substituting a with z
  Unifying: b and b
  Terms are identical, no substitution.

Final Unification Result: {'x': 'y', 'a': 'z'}

Example 2: Unifying P(x, f(Y)) and P(Z, f(a))

Unifying: ('P', ['x', ('f', ['Y'])]) and ('P', ['Z', ('f', ['a'])])
  Unifying: x and Z
  Unifying variable x with Z
    Substituting x with Z
  Unifying: ('f', ['Y']) and ('f', ['a'])
    Unifying: Y and a
    Unifying variable a with Y
      Substituting a with Y

Final Unification Result: {'x': 'Z', 'a': 'Y'}


In [2]:
class UnificationError(Exception):
    pass

def unify(expr1, expr2, substitutions=None):
    if substitutions is None:
        substitutions = {}

    # If both expressions are identical, return current substitutions
    if expr1 == expr2:
        return substitutions

    # If the first expression is a variable
    if is_variable(expr1):
        return unify_variable(expr1, expr2, substitutions)

    # If the second expression is a variable
    if is_variable(expr2):
        return unify_variable(expr2, expr1, substitutions)

    # If both expressions are compound expressions
    if is_compound(expr1) and is_compound(expr2):
        if expr1[0] != expr2[0] or len(expr1[1:]) != len(expr2[1:]):
            raise UnificationError("Expressions do not match.")
        return unify_lists(expr1[1:], expr2[1:], unify(expr1[0], expr2[0], substitutions))

    # If expressions are not compatible
    raise UnificationError(f"Cannot unify {expr1} and {expr2}.")

def unify_variable(var, expr, substitutions):
    if var in substitutions:
        return unify(substitutions[var], expr, substitutions)
    elif occurs_check(var, expr, substitutions):
        raise UnificationError(f"Occurs check failed: {var} in {expr}.")
    else:
        substitutions[var] = expr
        return substitutions

def unify_lists(list1, list2, substitutions):
    for expr1, expr2 in zip(list1, list2):
        substitutions = unify(expr1, expr2, substitutions)
    return substitutions

def is_variable(term):
    return isinstance(term, str) and term[0].islower()

def is_compound(term):
    return isinstance(term, (list, tuple)) and len(term) > 0

def occurs_check(var, expr, substitutions):
    if var == expr:
        return True
    elif is_compound(expr):
        return any(occurs_check(var, sub, substitutions) for sub in expr)
    elif expr in substitutions:
        return occurs_check(var, substitutions[expr], substitutions)
    return False

# Example usage
try:
    expr1 = ("f", "x", ("g", "y"))
    expr2 = ("f", "a", ("g", "b"))
    result = unify(expr1, expr2)
    print("Unified substitutions:", result)
except UnificationError as e:
    print("Unification failed:", e)

Unified substitutions: {'x': 'a', 'y': 'b'}
